<a href="https://colab.research.google.com/github/LuanFonsecaD/Algoritmos/blob/main/8-Puzzle_Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import copy
import numpy as np

#Calcula o índice do número na no tabuleiro (matriz)
def disposicao(jogo, numero):
    for i, linha in enumerate(jogo):
        for j, elemento in enumerate(linha):
            if elemento == numero:
                return (i, j)
    return None

#Calcula a heurística H pela distância manhattan entre as matrizes
def heuristica(est_atual, est_objetivo):
    hdist = 0
    for i in range(3):
        for j in range(3):
            num = est_atual[i][j]
            #O zero somente representa uma posição vazia.Não conta como número
            if num != 0:
                # encontra a posição do elemento na matriz objetivo
                posicao = disposicao(est_atual, num);
                posicaoObjetivo = disposicao(est_objetivo, num);
                # calcula a distância as posições atual e objetivo
                dist = (abs(posicaoObjetivo[0]-posicao[0]) + abs(posicaoObjetivo[1]-posicao[1]))
                #Soma a distancia à h 
                hdist += dist
    return hdist

def expandir_no(jogo):    
    movimentos = []
    i, j = None, None
    # Encontra a posição do espaço vazio (0)
    for x in range(3):
        for y in range(3):
            if jogo[x][y] == 0:
                i, j = x, y
                break
        if i is not None and j is not None:
            break
    # Gera as jogadas possíveis
    if i > 0:
        jogada = [linha.copy() for linha in jogo]
        jogada[i][j], jogada[i-1][j] = jogada[i-1][j], jogada[i][j]
        movimentos.append(("Cima", jogada))
    if i < 2:
        jogada = [linha.copy() for linha in jogo]
        jogada[i][j], jogada[i+1][j] = jogada[i+1][j], jogada[i][j]
        movimentos.append(("Baixo", jogada))
    if j > 0:
        jogada = [linha.copy() for linha in jogo]
        jogada[i][j], jogada[i][j-1] = jogada[i][j-1], jogada[i][j]
        movimentos.append(("Esquerda", jogada))
    if j < 2:
        jogada = [linha.copy() for linha in jogo]
        jogada[i][j], jogada[i][j+1] = jogada[i][j+1], jogada[i][j]
        movimentos.append(("Direita", jogada))
    return movimentos

class No:

    #Inicialização da classe Nó
    def __init__(self, paramConfiguracao, paramPai=None, paramAcao=None, g=0, h=0):
        self.configuracao = paramConfiguracao
        self.pai = paramPai
        self.acao = paramAcao
        self.g = g
        self.h = h

    #Função de avaliação do Nó f(n) = g + h   (Custo + Heurística) 
    def funcaoAvaliacao(self):
        return self.g + self.h
#Função que varre os Nós da lista e verifica se já existe um Nó com a mesma configuração
def existeConfigNaLista(config, fronteira):
    for no in fronteira:
        if no.configuracao == config:
            return no
    return None

#Executa o algoritmo de busca A* 
iteracao = 1 
def buscaAestrela(paramConfigInicial, paramConfigObjetivo, inImprimirFronteira):   
    global iteracao
    lstNosExpandidos = []
    noRaiz = No(paramConfigInicial, None, None, 0, heuristica(paramConfigInicial, paramConfigObjetivo))    
    #Cria a fronteira com o nó raiz
    fronteira = [noRaiz]    
    #Itera a fronteira buscando a melhor solução    
    while fronteira:  
        if (inImprimirFronteira == True):
            # Imprime a fronteira da iteração atual na tela                 
            print("Iteração " + str(iteracao) + ":\n")
            for noFilho in fronteira:
                #Formata a string de exibição do Nó raiz com a configuração   
                configuracao = np.array(noFilho.configuracao)
                strConfiguracao = ""
                for linha in configuracao:
                    strConfiguracao = strConfiguracao + str(linha) + "\n"
                print(strConfiguracao)            
                print(f'=> g: {noFilho.g}, h: {noFilho.h}, avaliacao: {noFilho.funcaoAvaliacao()}\n')        
        #Seleciona o menor nó da fronteira considerando f(n) = g(n) + h(n)
        melhorNo = min(fronteira, key=lambda noFronteira: noFronteira.funcaoAvaliacao())
        if (inImprimirFronteira == True):
            print(f'Melhor Nó=> g: {melhorNo.g}, h: {melhorNo.h}, avaliacao: {melhorNo.funcaoAvaliacao()}\n')         
        # Guarda os nos expandidos    
        lstNosExpandidos.append(melhorNo)
        #Remove o nó da fronteira
        fronteira.remove(melhorNo)        
        #Verifica se o nó escolhido é a solução        
        if np.array_equal(melhorNo.configuracao,paramConfigObjetivo):
            return melhorNo
        iteracao = iteracao + 1
        #Gera novos Nós a partir do melhor Nó avaliado e reordena a fronteira pela função de avaliação de cada Nó
        listaNovosNos = expandir_no(melhorNo.configuracao)
        for acao, configuracao in listaNovosNos:
            g = melhorNo.g + 1 
            h = heuristica(configuracao, paramConfigObjetivo)
            noFilho = No(configuracao, melhorNo, acao, g, h)          
            #Se já existe na fronteira atualiza o custo e não inclui filho
            noFronteiraIgual = existeConfigNaLista(noFilho.configuracao, fronteira)
            if noFronteiraIgual is not None:                                
                if noFilho.g >= noFronteiraIgual.g:
                    noFronteiraIgual.g = noFilho.g                    
                    noFronteiraIgual.pai = melhorNo
                    noFronteiraIgual.acao = acao                                    
            else:
                # Não inclui um No que já foi expandido na fronteira
                noJaExpandido = existeConfigNaLista(noFilho.configuracao, lstNosExpandidos)
                if (noJaExpandido is None):
                    fronteira.append(noFilho) 
                
                
    return None

#Mostra solução escolhida pelo algoritmo 
def exibirSolucao(paramNo):
    caminho = []  
    global iteracao   

    print("\nCaminho da solução encontrada:\n")
    #Caminha pela solução no sentido bottom up do Nó encontrado até a raiz
    while paramNo.pai:        
        #Formata a string do Nó com a ação tomada e a configuração
        configuracao = np.array(paramNo.configuracao)
        strConfiguracao = ""
        for linha in configuracao:
            strConfiguracao = strConfiguracao + str(linha) + "\n"        
        caminho.append("Ação " + paramNo.acao + " g(n)= " + str(paramNo.g) + " h(n)= " + str(paramNo.h) + " f(n)= " + str(paramNo.funcaoAvaliacao()) + '\n' + strConfiguracao)
        paramNo = paramNo.pai
    #Formata a string de exibição do Nó raiz com a configuração   
    configuracao = np.array(paramNo.configuracao)
    strConfiguracao = ""
    for linha in configuracao:
        strConfiguracao = strConfiguracao + str(linha) + "\n"
    caminho.append(strConfiguracao) 
    #Itera a o caminho de trás para frente já que o primeiro Nó é a solução (Nó folha) 
    for i in range(len(caminho)-1, -1, -1):
        #Imprime a fronteira na tela
        print(caminho[i])

    print("\nNúmero de Iterações na fronteira:" + str(iteracao))

# Conta a quantidade d einversões
def getTotalInversoes(pLista):
    qtdInversoes = 0
    valorVazio = 0
    for i in range(0, 9):
        for j in range(i + 1, 9):
            if pLista[j] != valorVazio and pLista[i] != valorVazio and pLista[i] > pLista[j]:
                qtdInversoes += 1
    return qtdInversoes 
     
# Verifica se tem solução
def checaSeSolucaoPossivel(pMatriz) :
 
    # Conta as inversões
    total_inversoes = getTotalInversoes([j for sub in pMatriz for j in sub])
 
    # return true if inversion count is even.
    return (total_inversoes % 2 == 0)

#Configuração inicial do jogo
configuracaoInicial = [[1, 8, 2], [0, 4, 3], [7, 6, 5]]
#configuracaoInicial = [[8, 1, 2], [0, 4, 3], [7, 6, 5]]
#Ordenação objetivo do jogo considerando as posições no tabuleiro.    
configuracaoObjetivo = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
#Checa se tem uma solução
inTemSolucao = checaSeSolucaoPossivel(configuracaoInicial)

if (inTemSolucao == True) :
    #Executa o algoritmo A* e retorna a melhor solução para chegar ao Nó com a mesma configuração do objetivo
    noSolucao = buscaAestrela(configuracaoInicial, configuracaoObjetivo, True)
    #Exibe Solução encontrada
    exibirSolucao(noSolucao)
else:
    print("Não há solução possível para essa configuração :-(")




Iteração 1:

[1 8 2]
[0 4 3]
[7 6 5]

=> g: 0, h: 9, avaliacao: 9

Melhor Nó=> g: 0, h: 9, avaliacao: 9

Iteração 2:

[0 8 2]
[1 4 3]
[7 6 5]

=> g: 1, h: 10, avaliacao: 11

[1 8 2]
[7 4 3]
[0 6 5]

=> g: 1, h: 10, avaliacao: 11

[1 8 2]
[4 0 3]
[7 6 5]

=> g: 1, h: 8, avaliacao: 9

Melhor Nó=> g: 1, h: 8, avaliacao: 9

Iteração 3:

[0 8 2]
[1 4 3]
[7 6 5]

=> g: 1, h: 10, avaliacao: 11

[1 8 2]
[7 4 3]
[0 6 5]

=> g: 1, h: 10, avaliacao: 11

[1 0 2]
[4 8 3]
[7 6 5]

=> g: 2, h: 7, avaliacao: 9

[1 8 2]
[4 6 3]
[7 0 5]

=> g: 2, h: 7, avaliacao: 9

[1 8 2]
[4 3 0]
[7 6 5]

=> g: 2, h: 9, avaliacao: 11

Melhor Nó=> g: 2, h: 7, avaliacao: 9

Iteração 4:

[0 8 2]
[1 4 3]
[7 6 5]

=> g: 1, h: 10, avaliacao: 11

[1 8 2]
[7 4 3]
[0 6 5]

=> g: 1, h: 10, avaliacao: 11

[1 8 2]
[4 6 3]
[7 0 5]

=> g: 2, h: 7, avaliacao: 9

[1 8 2]
[4 3 0]
[7 6 5]

=> g: 2, h: 9, avaliacao: 11

[0 1 2]
[4 8 3]
[7 6 5]

=> g: 3, h: 8, avaliacao: 11

[1 2 0]
[4 8 3]
[7 6 5]

=> g: 3, h: 6, avaliacao: 9

Melhor Nó